# Real world Data

## Data source 

Basically, the dataset tree is the following:

- Region
    - Country
        - Industry
            - Client 
                - Products
                
## Variables name

|  Variable name        |  Type   |
| --------------------- | ------- |
| ID                    |  object |
| Country_name          | object  |
| country_ref           | object  |
| Region                | object  |
| industry              | object  |
| A_reference           | float64 |
| A_market              | float64 |
| B_reference           | float64 |
| B_market              | float64 |
| C_reference           | float64 |
| C_market              | float64 |
| D_reference           | float64 |
| D_market              | float64 |
| E_reference           | float64 |
| E_market              | float64 |
| Total_reference       | float64 |
| Total_market          | float64 |

Note that, the market includes the reference

In [1]:
import pandas as pd
#import plotly.express as px
import numpy as np

import seaborn as sns

cm = sns.light_palette("green", as_cmap=True)

C:\Users\PERNETTH\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
path = r"C:\Users\PERNETTH\Documents\Projects\PythonTeaching\Lectures\05_Pandas\dataPandasClass.gz"

In [3]:
df_final = pd.read_csv(path)
df_final.head()

,ID,Country_name,country_ref,Region,industry,A_reference,A_market,B_reference,B_market,C_reference,C_market,D_reference,D_market,E_reference,E_market,Total_reference,Total_market
0,Customer 620,Australia,AUS,Asia Pacific,A,0.0,0.007258,0.0,0.010424,0.0,0.021051,0.0,0.001752,0.0,0.005433,0.0,0.021454
1,Customer 1070,Australia,AUS,Asia Pacific,A,0.0,0.009222,0.0,0.018054,0.0,0.021051,0.0,0.009690,0.0,0.008571,0.0,0.023859
2,Customer 25,Australia,AUS,Asia Pacific,A,0.0,0.012397,0.0,0.012396,0.0,0.002533,0.0,0.007179,0.0,0.005884,0.0,0.015037
3,Customer 370,Australia,AUS,Asia Pacific,B,0.0,0.012402,0.0,0.032244,0.0,0.009523,0.0,0.016637,0.0,0.016271,0.0,0.033501
4,Customer 597,Australia,AUS,Asia Pacific,C,0.0,0.014409,0.0,0.015047,0.0,0.010695,0.0,0.009601,0.0,0.010526,0.0,0.018923


In [ ]:
list(df_final)

In [ ]:
df_final.sort_values(by =['ID','Country_name']).head()

In [ ]:
df_final.sort_values(by =['ID','Country_name'],
                    ascending =[True, False]).head()

In [ ]:
df_final.filter(like = "_").head()

In [ ]:
df_final.filter(regex = "A|a").head(5)

In [ ]:
df_final.filter(regex = "A|a").tail(5)

In [ ]:
df_final.shape

In [ ]:
df_final.shape[0]

In [ ]:
df_final.shape[1]

## List unique variables

- customer
- country
- industry


In [ ]:
df_final[['ID', 'Region', 'Country_name', 'industry']].nunique()

In [ ]:
df_final['Region'].unique()

In [ ]:
df_final.loc[lambda x: x['Region'].isin(['Asia Pacific'])]['Country_name'].nunique()

In [ ]:
df_final.loc[lambda x: x['Region'].isin(['Asia Pacific'])]['Country_name'].unique()

In [ ]:
df_final.loc[lambda x: x['Region'].isin(['Asia Pacific', 'Northern Europe'])]

In [ ]:
list_region =  df_final['Region'].unique()
list_region

In [ ]:
df_final.loc[lambda x: x['Region'].isin(['Asia Pacific'])]['Country_name'].unique()

In [ ]:
df_final.loc[lambda x: x['Region'].isin(list_region)]['Country_name'].unique()

## List country by region

In [ ]:
df_final['Region']

In [ ]:
for r in df_final['Region'].sort_values().unique():

    countries = df_final.loc[lambda x :x['Region'].isin(
        [r])]['Country_name'].unique()

    print('\n Region name {} \n {}'.format(r, countries))

### Average of product A (A_reference) by region

In [ ]:
for r in df_final['Region'].sort_values().unique():

    countries = df_final.loc[lambda x :x['Region'].isin(
        [r])]['A_reference'].mean()

    print('\n Region name {} \n {:.2%}'.format(r, countries))

In [ ]:
df_final.groupby('Region')['A_reference'].mean()

In [ ]:
df_final.filter(regex = 'A').mean()

In [ ]:
df_final.groupby('Region')[['A_reference', 'A_market']].mean()

In [ ]:
df_final.filter(regex = 'Region|A|B').groupby('Region').mean()

In [ ]:
(df_final
 .filter(regex = 'Region|A|B')# 1 -> filter
 .groupby('Region')# 2 -> group by region
 .mean()# 3 -> compute mean
 .assign( mean_a_b_ref = lambda x: x['A_reference'] - 
        x['B_reference'])# 4 -> compute mean difference
)

In [ ]:
(df_final
 .filter(regex = 'Region|A|B')# 1 -> filter
 .groupby('Region')# 2 -> group by region
 .mean()# 3 -> compute mean
 .assign( mean_a_b_ref = lambda x: x['A_reference'] - 
        x['B_reference'])# 4 -> compute mean difference
 .style
 .format('{:.2%}')
 .background_gradient(cmap='viridis', low=.5, high=0)
 #.to_excel('styled.xlsx', engine='openpyxl')
 #.to_csv('mean_A_B.csv')
)

## Aggregation

In [ ]:
list(df_final)

In [ ]:
cd "C:\Users\PERNETTH\Documents\Projects\InseeInpi_matching" && jupyter notebook

In [ ]:
agg_1 = df_final.groupby(['industry']).agg(

    {
        'A_market': ['min', 'max', 'mean'],
        'B_market': ['min', 'max', 'mean'],
    }
)
agg_1#.reset_index()

In [ ]:
agg_2 = df_final.groupby(['Region','industry']).agg(

    {
        'A_market': ['min', 'max', 'mean'],
        'B_market': ['min', 'max', 'mean'],
    }
)
agg_2

In [ ]:
agg_2.loc[('Americas')]

In [ ]:
agg_2.loc[('Americas', 'A')]

In [ ]:
df_final.groupby(['Region','industry'])[['A_market', 'B_market']].describe()

In [ ]:
agg_3 = df_final.groupby(['Region','Country_name','industry'])[['A_market', 'B_market']].describe()
agg_3

In [ ]:
(agg_3
 .loc[lambda x :
      x.index.get_level_values('Region').isin(['Americas',
                                              'Southern Europe'])]
)

In [ ]:
(agg_3
 .loc[lambda x :
      x.index.get_level_values('industry').isin(['A','B'])]
)

In [ ]:
(agg_3
 .loc[lambda x:
      (x.index.get_level_values('Region').isin(['Americas']))
      & (x.index.get_level_values('industry').isin(['A', 'B']))
      ]
 )

In [ ]:
(df_final
 .groupby(['Region', 'Country_name', 'industry'])
 .agg(

     {
         'A_market': ['min', 'max', 'mean'],
         'B_market': ['min', 'max', 'mean']
     }
 )
 .loc[lambda x:
         (x.index.get_level_values('Region').isin(['Americas']))
         & (x.index.get_level_values('industry').isin(['A', 'B']))
         ]
 )

## Index

In [ ]:
list(df_final)

In [ ]:
df_final.head()

In [ ]:
df_final.loc[:5]

In [ ]:
df_index = df_final.set_index(['ID',
                    'Country_name',
                    'Region',
                    'industry'])
df_index.head()

In [ ]:
#df_index.loc[0]

In [ ]:
df_index.loc[('Customer 25')]#.mean()

In [ ]:
df_index.loc[('Customer 25','Belgium')]

In [ ]:
df_index.loc[('Customer 25',
              slice(None),
              slice(None), 'A')].head()

In [ ]:
df_index.loc[('Customer 25',
              slice(None),
              slice(None), 'A'), ['A_reference', 'B_reference']].head()

In [ ]:
df_index.xs('Belgium',
            level='Country_name',
            axis=0, drop_level=False).head()

In [ ]:
df_index.xs(('Belgium'),
            level=('Country_name'),
            axis=0, drop_level=False).head()

In [ ]:
df_index.loc[('Customer 25'), 'A_reference']

# Top level

## Objectives

In this notebook, you need to:

- Print the country name by region
- Compute the market share for the reference by:
    - Product
    - Industry
    - Region
- Industry Market share by product
- Region Market share by product
- Region Market share by industry

For all the above computation compute the market share of `reference`

- Remove Region-industry with no revenue
- Compute Euclidean Distance by Region-industry-product
- Filter the data by Region-industry-product

**theShare of wallet :Revenue to potential**

$$\text{Share of wallet} = \sum  \text{Revenue Reference} / \sum \text{Total Market} $$

In [4]:
reference = [
    "A_reference",
    "B_reference",
    "C_reference",
    "D_reference",
    "E_reference",
]
market = [
    "A_market",
    "B_market",
    "C_market",
    "D_market",
    "E_market",
]

In [ ]:
df_final.head(2)

## Industry Market size

In [ ]:
(df_final
 .groupby('industry')[['A_reference', 'A_market']]
 .sum()
 .assign(
     sow = lambda x: x['A_reference']/ x['A_market']
 )
 .sort_values(by = 'sow')
 .style
 .bar(subset=['sow'], color='#d65f5f')
 .format("{:.2%}", subset=['sow'])
)
)

## Region Market size

In [ ]:
(df_final
 .groupby('Region')[['A_reference', 'A_market']]
 .sum()
 .assign(
     sow = lambda x: x['A_reference']/ x['A_market']
 )
 .sort_values(by = 'sow')
 .style
 .bar(subset=['sow'], color='#d65f5f')
 .format("{:.2%}", subset=['sow'])
)

In [ ]:
df_final.groupby('Region')['ID'].nunique()

In [ ]:
df_final.tail(25)

In [ ]:
(df_final
 .assign(sow = lambda x: x['A_reference']/ x['A_market'])
 .groupby(['Country_name', 'industry'])['sow']
 .describe()
 .loc[lambda x: 
      (x.index.get_level_values('Country_name').isin(['France']))
     & (x.index.get_level_values('industry').isin(['A']))
     ]
)

In [ ]:
(df_final
 .loc[lambda x: 
      (x['A_reference'] != 0)
      & (x['A_market'] != 0)
      & x['Country_name'].isin(['France'])
      & x['industry'].isin(['D'])
     ]
 #.assign(sow = lambda x: x['A_reference']/ x['A_market'])
 #.groupby(['Country_name', 'industry'])['sow']
 #.describe()
 #.loc[lambda x: 
 #     (x.index.get_level_values('Country_name').isin(['France']))
 #    & (x.index.get_level_values('industry').isin(['C']))
 #    ]
)

In [ ]:
(df_final
 .loc[lambda x :x['ID'].isin(['Customer 46'])]
.assign(sow = lambda x: x['A_reference']/ x['A_market'])
.reindex(columns = ['ID','industry', 'Country_name', 'A_reference', 'A_market', 'sow'])
.sort_values('sow', ascending = False))

# Next level

In [21]:
(df_final[['industry'] + reference + market].groupby('industry')
.sum().head())

,A_reference,B_reference,C_reference,D_reference,E_reference,A_market,B_market,C_market,D_market,E_market
industry,,,,,,,,,,
A,4.907946,0.253223,1.020902,10.445549,0.461802,52.405622,31.906430,24.505209,19.678371,16.777328
B,2.210468,0.249729,0.795853,3.545234,0.393175,19.570229,13.097126,9.195902,7.722845,6.785848
C,2.902790,0.250033,0.638045,5.092406,0.376686,27.570648,19.220952,13.510517,10.936563,8.594297
D,0.932114,0.101370,0.094534,1.590237,0.217327,9.488114,6.047796,5.051537,3.569419,2.891857
E,5.232745,0.320306,1.314825,11.143459,0.474185,61.288900,39.793848,28.857113,20.745395,19.364490


In [17]:
step_1 = (df_final[['industry'] + reference + market]
.groupby('industry')
.sum()
.T
.reset_index()
 .assign(origin = lambda x: x['index'].str.extract(r"(_[^_]+$)"),
                product = lambda x: x['index'].str.extract(r"(^[^_]+(?=_))"))
)
step_1

industry,index,A,B,C,D,E,F,G,H,I,...,K,L,M,N,O,P,Q,R,origin,product
0,A_reference,4.907946,2.210468,2.902790,0.932114,5.232745,2.252117,2.677418,1.243388,0.979546,...,0.664858,1.553131,0.618725,0.911423,0.251024,0.000000,0.031190,0.044136,_reference,A
1,B_reference,0.253223,0.249729,0.250033,0.101370,0.320306,0.207141,0.355712,0.029682,0.052515,...,0.070875,0.072015,0.028072,0.009750,0.037574,0.000000,0.010367,0.005411,_reference,B
2,C_reference,1.020902,0.795853,0.638045,0.094534,1.314825,0.450612,0.430902,0.331000,0.128314,...,0.146820,0.324976,0.153012,0.057365,0.045982,0.000000,0.000000,0.014305,_reference,C
3,D_reference,10.445549,3.545234,5.092406,1.590237,11.143459,4.844611,5.215100,2.140190,1.674319,...,1.305611,4.215493,1.068068,1.762081,0.430486,0.000000,0.022291,0.121586,_reference,D
4,E_reference,0.461802,0.393175,0.376686,0.217327,0.474185,0.204973,0.499346,0.124389,0.074575,...,0.027251,0.184002,0.064696,0.032391,0.010517,0.000000,0.000000,0.023121,_reference,E
5,A_market,52.405622,19.570229,27.570648,9.488114,61.288900,20.369299,27.354353,10.857058,8.392135,...,5.971862,19.598920,4.775709,7.539815,1.951133,0.026133,0.242144,2.406654,_market,A
6,B_market,31.906430,13.097126,19.220952,6.047796,39.793848,14.971596,17.681935,6.903328,6.335236,...,4.559139,12.291928,3.901281,6.860973,1.401172,0.026943,0.192816,1.272574,_market,B
7,C_market,24.505209,9.195902,13.510517,5.051537,28.857113,8.537154,9.808075,5.468393,4.195441,...,2.557313,8.430008,2.011502,4.898695,1.031104,0.010029,0.101499,0.789227,_market,C
8,D_market,19.678371,7.722845,10.936563,3.569419,20.745395,8.629699,9.673067,4.113519,3.931542,...,2.636369,7.300546,2.289063,5.545575,0.859661,0.019798,0.110235,0.593220,_market,D
9,E_market,16.777328,6.785848,8.594297,2.891857,19.364490,7.504141,8.296581,3.568844,3.310806,...,1.973700,6.191475,1.908590,5.238381,0.835774,0.029784,0.051864,0.412629,_market,E


In [18]:
step_2 = (step_1
          .drop(columns='index')
          .sort_values(by="product")
          .set_index(['product', 'origin'])
          )
step_2

industry                    A          B          C         D          E  \
product origin                                                             
A       _reference   4.907946   2.210468   2.902790  0.932114   5.232745   
        _market     52.405622  19.570229  27.570648  9.488114  61.288900   
B       _reference   0.253223   0.249729   0.250033  0.101370   0.320306   
        _market     31.906430  13.097126  19.220952  6.047796  39.793848   
C       _reference   1.020902   0.795853   0.638045  0.094534   1.314825   
        _market     24.505209   9.195902  13.510517  5.051537  28.857113   
D       _reference  10.445549   3.545234   5.092406  1.590237  11.143459   
        _market     19.678371   7.722845  10.936563  3.569419  20.745395   
E       _reference   0.461802   0.393175   0.376686  0.217327   0.474185   
        _market     16.777328   6.785848   8.594297  2.891857  19.364490   

industry                    F          G          H         I          J  \
product origin                                                             
A       _reference   2.252117   2.677418   1.243388  0.979546   1.844073   
        _market     20.369299  27.354353  10.857058  8.392135  23.596566   
B       _reference   0.207141   0.355712   0.029682  0.052515   0.099365   
        _market     14.971596  17.681935   6.903328  6.335236  12.638108   
C       _reference   0.450612   0.430902   0.331000  0.128314   0.481378   
        _market      8.537154   9.808075   5.468393  4.195441  11.065682   
D       _reference   4.844611   5.215100   2.140190  1.674319   3.771860   
        _market      8.629699   9.673067   4.113519  3.931542   7.763102   
E       _reference   0.204973   0.499346   0.124389  0.074575   0.295841   
        _market      7.504141   8.296581   3.568844  3.310806   6.473929   

industry                   K          L         M         N         O  \
product origin                                                          
A       _reference  0.664858   1.553131  0.618725  0.911423  0.251024   
        _market     5.971862  19.598920  4.775709  7.539815  1.951133   
B       _reference  0.070875   0.072015  0.028072  0.009750  0.037574   
        _market     4.559139  12.291928  3.901281  6.860973  1.401172   
C       _reference  0.146820   0.324976  0.153012  0.057365  0.045982   
        _market     2.557313   8.430008  2.011502  4.898695  1.031104   
D       _reference  1.305611   4.215493  1.068068  1.762081  0.430486   
        _market     2.636369   7.300546  2.289063  5.545575  0.859661   
E       _reference  0.027251   0.184002  0.064696  0.032391  0.010517   
        _market     1.973700   6.191475  1.908590  5.238381  0.835774   

industry                   P         Q         R  
product origin                                    
A       _reference  0.000000  0.031190  0.044136  
        _market     0.026133  0.242144  2.406654  
B       _reference  0.000000  0.010367  0.005411  
        _market     0.026943  0.192816  1.272574  
C       _reference  0.000000  0.000000  0.014305  
        _market     0.010029  0.101499  0.789227  
D       _reference  0.000000  0.022291  0.121586  
        _market     0.019798  0.110235  0.593220  
E       _reference  0.000000  0.000000  0.023121  
        _market     0.029784  0.051864  0.412629

In [27]:
step_3 = (step_2.groupby(level=0)
          .apply(lambda x: x.shift(1) / x)
          .dropna()
          )
step_3

,industry,A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R
product,origin,,,,,,,,,,,,,,,,,,
A,_market,0.093653,0.112951,0.105286,0.098240,0.085378,0.110564,0.097879,0.114523,0.116722,0.078150,0.111332,0.079246,0.129557,0.120881,0.128655,0.0,0.128809,0.018339
B,_market,0.007936,0.019067,0.013008,0.016761,0.008049,0.013836,0.020117,0.004300,0.008289,0.007862,0.015546,0.005859,0.007196,0.001421,0.026816,0.0,0.053766,0.004252
C,_market,0.041661,0.086544,0.047226,0.018714,0.045563,0.052782,0.043933,0.060530,0.030584,0.043502,0.057412,0.038550,0.076068,0.011710,0.044595,0.0,0.000000,0.018125
D,_market,0.530814,0.459058,0.465631,0.445517,0.537153,0.561388,0.539136,0.520282,0.425868,0.485870,0.495231,0.577422,0.466596,0.317745,0.500762,0.0,0.202217,0.204960
E,_market,0.027525,0.057940,0.043830,0.075151,0.024487,0.027315,0.060187,0.034854,0.022525,0.045697,0.013807,0.029719,0.033897,0.006183,0.012583,0.0,0.000000,0.056033


In [29]:
(step_3
 .droplevel('origin')
 .transpose()
 .assign(total_industry = lambda x: x.sum(1))
)

product,A,B,C,D,E,total_industry
industry,,,,,,
A,0.093653,0.007936,0.041661,0.530814,0.027525,0.701589
B,0.112951,0.019067,0.086544,0.459058,0.057940,0.735561
C,0.105286,0.013008,0.047226,0.465631,0.043830,0.674981
D,0.098240,0.016761,0.018714,0.445517,0.075151,0.654384
E,0.085378,0.008049,0.045563,0.537153,0.024487,0.700632
F,0.110564,0.013836,0.052782,0.561388,0.027315,0.765885
G,0.097879,0.020117,0.043933,0.539136,0.060187,0.761253
H,0.114523,0.004300,0.060530,0.520282,0.034854,0.734489
I,0.116722,0.008289,0.030584,0.425868,0.022525,0.603989


In [65]:
industrial_share = (df_final[['industry'] + reference + market]
 .groupby('industry')
 .sum()
 .transpose()
 .reset_index()
 .assign(origin = lambda x: x['index'].str.extract(r"(_[^_]+$)"),
        product = lambda x: x['index'].str.extract(r"(^[^_]+(?=_))"))
 .drop(columns = 'index')
 .sort_values(by = "product")
 .set_index(['product', 'origin'])
 .groupby(level = 0)
 .apply(lambda x: x.shift(1) / x )
 .dropna()
 .droplevel('origin')
 .transpose()
 .assign(total_industry = lambda x: x.sum(1))
 .sort_values(by ='total_industry')
 #.drop(columns = 'total_industry')
 .style
 .format("{:.2%}")
 .background_gradient(cmap=cm, subset = ['A', 'B', 'C', 'D', 'E'])
 .bar(subset=['total_industry'], color='#d65f5f')
)

## Industry Market size by product

In [ ]:
step_1 = (df_final[['industry'] + reference + market]
          .groupby('industry')
          .sum(axis=0)
          .transpose()
          .reset_index()
          .assign(origin = lambda x: x['index'].str.extract(r"(_[^_]+$)"),
                product = lambda x: x['index'].str.extract(r"(^[^_]+(?=_))"))
          )
step_1

In [ ]:
step_2 = (step_1.drop(columns = 'index')
 .sort_values(by = "product")
 .set_index(['product', 'origin'])
         )
step_2

In [ ]:
#step_2.unstack(-1)

In [ ]:
step_3 = (step_2.groupby(level=0)
          .apply(lambda x: x.shift(1) / x)
          .dropna()
          )
step_3

In [ ]:
(step_3.droplevel('origin')
 .transpose()
 .assign(total_industry = lambda x: x.sum(1))
)

In [ ]:
(df_final[['industry'] + reference + market]
 .groupby('industry')
 .sum(axis = 0)
 .transpose()
 .reset_index()
 .assign(origin = lambda x: x['index'].str.extract(r"(_[^_]+$)"),
        product = lambda x: x['index'].str.extract(r"(^[^_]+(?=_))"))
 .drop(columns = 'index')
 .sort_values(by = "product")
 .set_index(['product', 'origin'])
 .groupby(level = 0)
 .apply(lambda x: x.shift(1) / x )
 .dropna()
 .droplevel('origin')
 .transpose()
 .assign(total_industry = lambda x: x.sum(1))
 .sort_values(by ='total_industry')
 .drop(columns = 'total_industry')
 .style
 .format("{:.2%}")
 .background_gradient(cmap=cm)
)

## Region Market size by industry

In [40]:
(df_final[['Region'] + reference + market]
 .groupby('Region')
 .sum()
 .transpose()
 .reset_index()
 .assign(origin = lambda x: x['index'].str.extract(r"(_[^_]+$)"),
        product = lambda x: x['index'].str.extract(r"(^[^_]+(?=_))"))
 .drop(columns = 'index')
 .sort_values(by = "product")
 .set_index(['product', 'origin'])
 .groupby(level = 0)
 .apply(lambda x: x.shift(1) / x )
 .dropna()
 .droplevel('origin')
 .transpose()
 .assign(total_industry = lambda x: x.sum(1))
 .sort_values(by ='total_industry')
 #.drop(columns = 'total_industry')
 .style
 .format("{:.2%}")
 .background_gradient(cmap=cm, subset = ['A', 'B', 'C', 'D', 'E'])
 .bar(subset=['total_industry'], color='#d65f5f')
)

product,A,B,C,D,E,total_industry
Region,,,,,,
Asia Pacific,4.30%,0.27%,0.79%,18.43%,1.57%,25.36%
Northern Europe,5.76%,1.36%,1.68%,30.65%,4.72%,44.17%
Americas,0.59%,0.04%,1.71%,44.47%,37.29%,84.09%
Central Europe,11.56%,0.86%,5.48%,66.25%,6.14%,90.28%
Middle East & Africa,22.43%,0.06%,13.98%,61.70%,0.13%,98.30%
Southern Europe,25.68%,4.96%,10.45%,61.54%,0.50%,103.13%


## Correlation

In [ ]:
df_final.corr()

## Euclidean Distance

## Definition Euclidean Distance

According to the Euclidean distance formula, the distance between two points in the plane with coordinates (x, y), and is given by:

$$\begin{aligned} d(\mathbf{p}, \mathbf{q})=d(\mathbf{q}, \mathbf{p}) &=\sqrt{\left(q_{1}-p_{1}\right)^{2}+\left(q_{2}-p_{2}\right)^{2}+\cdots+\left(q_{n}-p_{n}\right)^{2}} \\ &=\sqrt{\sum_{i=1}^{n}\left(q_{i}-p_{i}\right)^{2}} \end{aligned}$$

### Example 

Calculate the distance between two arrays.

We want to sum each column of the first array with all the columns of the second array

Take an example, we want to compute the Euclidean distance of this first array:
- [7.2, 3.6, 5.1, 2.5]: It has 4 columns and one row

we will compute the Euclidean distance with all the rows of the second array. The second array has also four columns. We are substracting each columns of the first and second array respecively 

Exemple with the first rows of the second array:
- [5.1, 3.5, 1.4, 0.2]

The Euclidean distance is $$\sqrt{((7.2 - 5.1)^2  + (3.6 - 3.5)^2 + (5.1 -1.4)^2 + (2.5 - 0.2)^2)}$$

We repeat for all the rows

## Data standardization

The value of distance measures is intimately related to the scale on which measurements are made.

In [41]:
import numpy as np
def euclideanDistance(vect_1, vect_2):
  
    """
      Compute the Euclidean distance 
    """
  
    sum_ = np.sum(np.power(vect_1 - vect_2, 2))
  
    euclideanD = np.sqrt(sum_)
    return euclideanD

In [44]:
vec_1 = [7.2, 3.6, 5.1, 2.5]
vec_2 = [5.1, 3.5, 1.4, 0.2]

euclideanDistance(vect_1 = np.array(vec_1),
                  vect_2= np.array(vec_2))

4.8373546489791295

In [95]:
reshape_data = (df_final[['industry'] + reference + market]
 .groupby('industry')
 .sum()
 .T
 .reset_index()
 .assign(origin = lambda x: x['index'].str.extract(r"(_[^_]+$)"),
                product = lambda x: x['index'].str.extract(r"(^[^_]+(?=_))"))
 .drop(columns = 'index')
 .sort_values(by = "product")
 .set_index(['product', 'origin'])
 .unstack(0)
 .T
 .reindex(columns =  ['_reference', '_market'])
)
reshape_data.head(20)

origin            _reference    _market
industry product                       
A        A          4.907946  52.405622
         B          0.253223  31.906430
         C          1.020902  24.505209
         D         10.445549  19.678371
         E          0.461802  16.777328
B        A          2.210468  19.570229
         B          0.249729  13.097126
         C          0.795853   9.195902
         D          3.545234   7.722845
         E          0.393175   6.785848
C        A          2.902790  27.570648
         B          0.250033  19.220952
         C          0.638045  13.510517
         D          5.092406  10.936563
         E          0.376686   8.594297
D        A          0.932114   9.488114
         B          0.101370   6.047796
         C          0.094534   5.051537
         D          1.590237   3.569419
         E          0.217327   2.891857

In [83]:
reshape_data.loc['A'].assign(share = lambda x: x['_reference']/x['_market'],
                            share_tot = lambda x: x['share'].sum())

origin,_reference,_market,share,share_tot
product,,,,
A,4.907946,52.405622,0.093653,0.701589
B,0.253223,31.906430,0.007936,0.701589
C,1.020902,24.505209,0.041661,0.701589
D,10.445549,19.678371,0.530814,0.701589
E,0.461802,16.777328,0.027525,0.701589


In [78]:
euclideanDistance(vect_1 = reshape_data.loc['A']['_reference'],
                  vect_2= reshape_data.loc['A']['_market'])

64.50510638781942

In [85]:
reshape_data.loc['B'].assign(share = lambda x: x['_reference']/x['_market'],
                            share_tot = lambda x: x['share'].sum())

origin,_reference,_market,share,share_tot
product,,,,
A,2.210468,19.570229,0.112951,0.735561
B,0.249729,13.097126,0.019067,0.735561
C,0.795853,9.195902,0.086544,0.735561
D,3.545234,7.722845,0.459058,0.735561
E,0.393175,6.785848,0.057940,0.735561


In [79]:
euclideanDistance(vect_1 = reshape_data.loc['B']['_reference'],
                  vect_2= reshape_data.loc['B']['_market'])

24.398698009395048

In [98]:
reshape_data.head()

origin            _reference    _market
industry product                       
A        A          4.907946  52.405622
         B          0.253223  31.906430
         C          1.020902  24.505209
         D         10.445549  19.678371
         E          0.461802  16.777328

In [99]:
(reshape_data
 .groupby(level=0)
 .apply(lambda x: euclideanDistance(x.iloc[:, 0],
                                    x.iloc[:, 1],
                                    )
        )
 .sort_values()
 )

industry
P     0.052806
Q     0.313871
O     2.566219
R     2.856895
M     6.373934
K     7.725209
I    11.248503
D    12.008633
N    12.479202
H    13.484093
L    24.213129
B    24.398698
F    26.061817
J    28.222638
G    32.829116
C    35.153703
A    64.505106
E    76.863981
dtype: float64

In [101]:
reshape_data.loc['E'].assign(share = lambda x: x['_reference']/x['_market'],
                            share_tot = lambda x: x['share'].sum())

origin,_reference,_market,share,share_tot
product,,,,
A,5.232745,61.288900,0.085378,0.700632
B,0.320306,39.793848,0.008049,0.700632
C,1.314825,28.857113,0.045563,0.700632
D,11.143459,20.745395,0.537153,0.700632
E,0.474185,19.364490,0.024487,0.700632


## region-industry

In [105]:
reshape_data = (df_final[['industry', 'Region'] + reference + market]
 .groupby(['Region','industry' ])
 .sum()
 .T
 .reset_index()
 .assign(origin = lambda x: x['index'].str.extract(r"(_[^_]+$)"),
                product = lambda x: x['index'].str.extract(r"(^[^_]+(?=_))"))
 .drop(columns = 'index')
 .sort_values(by = "product")
 .set_index(['product', 'origin'])
 .unstack(0)
 .T
 .reindex(columns =  ['_reference', '_market'])
)
(reshape_data.groupby(level = [0, 1])
 .apply(lambda x: euclideanDistance(x.iloc[:, 0],
                                    x.iloc[:, 1],
                                        )
            )
    .sort_values()).loc['Northern Europe']


industry
Q     0.041325
O     0.642453
R     1.010593
N     1.561087
M     2.303541
K     2.503222
I     3.573567
D     3.604048
H     4.108689
B     7.028910
J     7.367411
L     7.712704
F     8.568435
G    10.337025
C    11.543228
A    18.257462
E    23.148816
dtype: float64

# Temp

In [ ]:
 .unstack(level = 0)
 .T
 .reindex(columns =  ['_reference', '_market'])
 .groupby(level = 0)
 .apply(lambda x: euclideanDistance(x.iloc[:, 0],
                                    x.iloc[:, 1],
                                        )
            )

In [66]:
industrial_share

product,A,B,C,D,E,total_industry
industry,,,,,,
P,0.00%,0.00%,0.00%,0.00%,0.00%,0.00%
R,1.83%,0.43%,1.81%,20.50%,5.60%,30.17%
Q,12.88%,5.38%,0.00%,20.22%,0.00%,38.48%
N,12.09%,0.14%,1.17%,31.77%,0.62%,45.79%
I,11.67%,0.83%,3.06%,42.59%,2.25%,60.40%
D,9.82%,1.68%,1.87%,44.55%,7.52%,65.44%
J,7.82%,0.79%,4.35%,48.59%,4.57%,66.11%
C,10.53%,1.30%,4.72%,46.56%,4.38%,67.50%
K,11.13%,1.55%,5.74%,49.52%,1.38%,69.33%
